In [1]:
import sys
sys.path.append("clustering")

from utils import get_data_dir

In [2]:
import os
import ijson
import json
from tqdm import tqdm
import pandas as pd
from typing import Dict, Any

class DataPreprocessor:
    def __init__(self, data_dir: str):
        self.data_dir = data_dir
        self.abstracts_dir = os.path.join(data_dir, '2010s/classification_tasks/abstracts')
        self.labels_dir = os.path.join(data_dir, '2010s/classification_tasks/labels')
        os.makedirs(self.abstracts_dir, exist_ok=True)
        os.makedirs(self.labels_dir, exist_ok=True)

    def process_abstracts(self, mag_abstracts_path: str, cluster_df: pd.DataFrame,
                        paper_to_venue: Dict[str, Any]) -> None:
        """提取并保存论文摘要"""
        print("Filtering valid papers...")
        # 将cluster_df的index转换为集合，这样查找更快
        valid_venues = set(cluster_df.index)
        # 使用集合推导式，比列表推导式更快
        valid_pid = {pid for pid, vid in paper_to_venue.items() if vid in valid_venues}
        print(f"valid_pid: {len(valid_pid)}")

        # 获取文件大小用于进度条
        file_size = os.path.getsize(mag_abstracts_path)
        matched_count = 0
        
        # 准备输出路径
        output_path = os.path.join(self.abstracts_dir, 'paper_abstracts.json')
        # 使用批量处理提高写入效率
        batch = {}
        batch_size = 50000
        with open(mag_abstracts_path, 'r', encoding='utf-8') as input_file, \
            open(output_path, 'w', encoding='utf-8') as output_file:
            
            # 初始化进度条
            with tqdm(total=file_size, desc="Processing abstracts", 
                    unit="B", unit_scale=True) as pbar:
                
                parser = ijson.kvitems(input_file, "")  # 解析顶层键值对
                batch = []
                batch_size = 50000  # 增大批量写入规模
                counter = 0  # 控制进度条更新频率

                for paper_id, abstract in parser:
                    counter += 1
                    if counter % 10000 == 0:  # 每 1000 条更新一次进度条
                        pbar.update(input_file.tell() - pbar.n)

                    try:
                        paper_id = int(paper_id)
                        if paper_id in valid_pid:
                            batch.append({'PaperID': paper_id,
                                          'abstract':abstract})
                            matched_count += 1
                    except ValueError:
                        continue

                    if len(batch) >= batch_size:
                        output_file.write("\n".join(json.dumps(item) for item in batch) + "\n")
                        batch = []

                if batch:  # 写入剩余的批量数据
                    output_file.write("\n".join(json.dumps(item) for item in batch) + "\n")
        print(f"abstracts: {matched_count}")
        print(f"Saved to {output_path}")

    def extract_labels(self, cluster_df: pd.DataFrame, 
                      paper_to_venue: Dict[str, Any]) -> None:
        """为每种聚类方法提取并保存标签"""
        # 获取所有标签列
        label_columns = ['Scopus_label', 
                         'movMF_label', 
                         'kmeans_label', 
                         'skm_label',
                         'n2v_kmeans_label',
                         'cm_kmeans_label',
                         'bert_kmeans_label',
                         'gnn_kmeans_label',]  # 添加所需的标签列
        
        for label_col in label_columns:
            # 构建PaperID到标签的映射
            venue_to_label = cluster_df[label_col].to_dict()
            paper_labels = {
                pid: venue_to_label[vid] 
                for pid, vid in paper_to_venue.items() 
                if vid in venue_to_label
            }
            
            # 保存标签
            output_path = os.path.join(
                self.labels_dir, 
                f"{label_col.lower()}.json"
            )
            with open(output_path, 'w') as f:
                json.dump(paper_labels, f)

In [3]:
data_dir = get_data_dir()

MAG_paper_df = pd.read_parquet(os.path.join(get_data_dir(), "2010s","MAG_paper.parquet"))
cluster_df = pd.read_parquet(os.path.join(get_data_dir(), "2010s/clustering_results","cluster_df.parquet"))
paper_to_venue = MAG_paper_df['VenueID'].to_dict()
paper_to_venue

data_dir:  /home/lyuzhuoqi/projects/clustering/data
data_dir:  /home/lyuzhuoqi/projects/clustering/data
data_dir:  /home/lyuzhuoqi/projects/clustering/data


{2049933365: 80951755,
 2038148770: 119525064,
 2373199189: 2764482698,
 2289331308: 2764425571,
 2784227654: 77020770,
 3032776793: 2764659248,
 2359024845: 2764610422,
 2090765587: 2755434998,
 1982732226: 183492911,
 2385836617: 2764593375,
 2809173448: 2764863013,
 3140293050: 2622093537,
 2375443784: 2764554180,
 1987591524: 202680183,
 2071864527: 190066210,
 2040405307: 105183386,
 1986439752: 1980519,
 2153779033: 91660768,
 2078947818: 201081534,
 2024704296: 1135705775,
 2392142542: 2764554180,
 2051541198: 59479367,
 2058138512: 111727011,
 2357419162: 2764810065,
 3029664364: 2764697616,
 2022427143: 69957782,
 2143932387: 204931651,
 2353867085: 2764638780,
 2116213074: 115312524,
 2374396473: 2764819181,
 3020209503: 1191630588,
 2070406492: 6147291,
 2080860174: 152760256,
 1986865494: 113170167,
 2027925926: 1192710900,
 2089901363: 2915058201,
 2394031763: 2764453096,
 1992202857: 15484759,
 2089307567: 111155417,
 2080680783: 104917558,
 2357501125: 2764598783,
 20670

In [4]:
preprocessor = DataPreprocessor(data_dir)

In [13]:
preprocessor.process_abstracts(os.path.join(data_dir,'MAG_abstracts.json'), cluster_df, paper_to_venue)

Filtering valid papers...
valid_pid: 1915871


Processing abstracts: 100%|██████████████████████████████████████████████████▉| 177G/177G [25:15<00:00, 116MB/s]


abstracts: 1527417
Saved to /home/lyuzhuoqi/projects/clustering/data/2010s/classification_tasks/abstracts/paper_abstracts.json


In [5]:
preprocessor.extract_labels(cluster_df, paper_to_venue)

In [6]:
import json

def read_lines(file_path, n: int=2):
    """
    从 Flat JSON 文件中读取前两行并打印解析结果。
    
    Args:
        file_path (str): Flat JSON 文件路径（每行一个 JSON 对象）。
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        for idx, line in enumerate(f):
            if idx < n:  # 只处理前n行
                try:
                    parsed_line = json.loads(line.strip())  # 解析 JSON 对象
                    print(json.dumps(parsed_line, indent=4, ensure_ascii=False))  # 美化输出
                except json.JSONDecodeError as e:
                    print(f"解析失败：{e}, 原始数据：{line.strip()}")
            else:
                break

In [7]:
output_path = os.path.join(get_data_dir(), "2010s/classification_tasks/abstracts", "paper_abstracts.json")
read_lines(output_path, 20)

data_dir:  /home/lyuzhuoqi/projects/clustering/data
{
    "PaperID": 2055207790,
    "abstract": "Matrix metalloproteinase-13 (MMP-13), referred to as collagenase-3, is a proteolytic enzyme that plays a key role in degradation and remodelling of host extracellular matrix proteins. The objective of this study was to characterize the MMP-13 gene in channel catfish, and to determine its pattern of expression in various healthy tissues and during embryogenesis. Since MMP-13 has been shown to have importance in tissue remodelling and some pathological processes, we further studied its involvement in the defense responses of catfish after bacterial infection. The channel catfish MMP-13 cDNA contains an open reading frame of 1416 bp encoding 471 amino acids. Using RT-PCR analysis, MMP-13 was widely expressed in various health tissues. Using quantitative real-time PCR analysis, expression of MMP-13 gene was up-regulated by bacterial infection. During normal embryological development, MMP-13 ex